In [1]:
import pyomo.environ as pyo

def model():
    m = pyo.ConcreteModel()
    m.x1 = pyo.Var(within=pyo.NonNegativeReals)
    m.x2 = pyo.Var(within=pyo.NonNegativeReals)
    m.cost = pyo.Objective(rule=10*m.x1 + 7*m.x2, sense = pyo.maximize)
    m.c1 = pyo.Constraint(rule=m.x1 + 2 * m.x2 <= 10)
    m.c2 = pyo.Constraint(rule=3 * m.x1 + 4*m.x2 <= 40)
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    return m

max z = Cx
s.t.
 Ax <= B

Dual:
min w = π^T b
s.t
A^T π >= c

Reduced costs: coefficients of z
Relationship between reduced costs and optimal solution of the problem π
Lemma: When the simplex algorithms finishes, we have: ĉj = cj - Σ π Aij <= 0, π = cB^-1

In [2]:
m = model()
solver = pyo.SolverFactory("cbc")
solver.solve(m)
objective = pyo.value(m.cost)

A **shadow price** value is associated with each constraint of the model.
It is the instantaneous change in the objective value of the optimal 
solution obtained by changing the right hand side constraint by one unit.

In [3]:
shadow_price_c1 = m.dual[m.c1]

A **reduce cost** value is associated with each variable of the model.
It is the maount by which an objective function parameter would have
to improve before it would be possible for a corresponding variable
to assume a positive value in the optimal solution.

x2 is outside the basis, its value is 0. Its reduce cost is negative.

In [4]:
assert pyo.value(m.x2) == 0
reduce_cost_x2 = m.rc[m.x2]
assert reduce_cost_x2 <= 0

x1 is in the basis so its value is different from 0 and its reduce cost equals 0

In [5]:
assert pyo.value(m.x1) != 0
assert m.rc[m.x1] == 0

Check the objective increases when we add the shadow price to the right hand constraint of c1

In [6]:
increase_in_c1 = 2
m = model()
m.del_component(m.c1)
m.c1 = pyo.Constraint(rule=m.x1 + 2 * m.x2 <= 10 + increase_in_c1)
solver.solve(m)
assert pyo.value(m.cost) == objective + increase_in_c1 * shadow_price_c1

Check that the objective decrease by the reduce cost of x2
The reduce cost of a decision variable (i.e. value -13 for variable x2)
is equal to the shadow prive of the non-negativity constraint of the variable
(m.x2 >= 0)

In [7]:
m = model()
increase_in_x2 = 1
m.within_x2 = pyo.Constraint(rule=m.x2 >= increase_in_x2)
solver.solve(m)
assert pyo.value(m.cost) < objective
assert pyo.value(m.cost) == objective + increase_in_x2 * reduce_cost_x2

We can check that the objective of the dual is equal to the objective of the primal

In [8]:
m_dual = pyo.ConcreteModel()
m_dual.π1 = pyo.Var(within=pyo.NonNegativeReals)
m_dual.π2 = pyo.Var(within=pyo.NonNegativeReals)
m_dual.cost = pyo.Objective(rule=10*m_dual.π1 + 40*m_dual.π2, sense = pyo.minimize)
m_dual.c1 = pyo.Constraint(rule=m_dual.π1 + 3 * m_dual.π2 >= 10)
m_dual.c2 = pyo.Constraint(rule=2 * m_dual.π1 + 4*m_dual.π2 >= 7)
solver.solve(m_dual)

assert pyo.value(m_dual.cost) == objective